# Simulate Asset Price Evolutions and Reprice Risky up-and-out Call Option

Group 6-A 

Members:

1. Md. Shafil Hosain (shafil430@yahoo.com)

2. Si Jie Lim (limsijie93@gmail.com)

3. Quang Vinh Dang (dqvinh87@gmail.com)

4. Wei Hao Lew (lewweihao93@hotmail.com)

5. Philip ZF Chen (philipchen619@gmail.com)

# Question 1

In this project, we will be using the LIBOR Forward Market Model(LFMM) to simulate the forward rates.

First, we import the necessary libraries needed for the code.

In [1]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
import functools
import math

Next, we define the list of parameter values needed.

In [2]:
L = 150                 #barrier limit
S_0 = 100               #starting share value
K = 100                 #strike price
V_0 = 200               #starting firm value
D = 175
gamma = 0.75
debt = 175              #firm debt
sigma_s = 0.30          #share volatility
sigma_v = 0.25          #firm value volatility
correlation = 0.2       #correlation between share and firm
recovery_rate = 0.25    #firm recovery rate
n_simulations = 100000
dt = 1/12               #expiry time of option / monthly simulations for a year
frequency = 12          #monthly simulations for a year
T = 1                   #expiry time of option

## Calibration

We are first going to calibrate the LFMM using the observed zero coupon bond prices. 

The jth forward rate at time t can be defined as

$F_j(t) = \frac{P_j(t)-P_{j+1}(t)}{\delta_t P_{j+1}(t)}$

We are going to use this formula to calibrate and obtain the initial forward rates, which will be used to simulate future rates as in the LFMM model.

In [3]:
def bondPricesToInitialForwardRates(bond_prices, n_steps):
    return np.ones([n_simulations, n_steps - 1]) * (bond_prices[:-1] - bond_prices[1:])/(dt * bond_prices[1:])

In [4]:
bond_prices = np.array([100, 99.38, 98.76, 98.15, 97.54, 96.94, 96.34, 95.74, 95.16, 94.57, 93.99, 93.42, 92.85])
n_steps = len(bond_prices) 

forward_rates = bondPricesToInitialForwardRates(bond_prices, n_steps)

## Interest Rate - LIBOR Forward Market Model (LFMM)

The LFMM says that the forward rate evolves according to the following formula

$F_j(t_i) = F_j(t_{i-1})exp[(\mu_j(t_{i-1})-\frac{1}{2}\sigma^2)\delta_{i-1} + \sigma_j\sqrt{\delta_{i-1}}Z_i]$

where

$\mu_j(t_{i-1}) \ \sum_{k=i}^j \frac{\delta_k F_j(t_{i-1})\sigma_k\sigma_j}{1+\delta_kF_j(t_{i-1})}$

which can be written as

In [5]:
def getMu(forward_rates, delta, sigma_j, sigma_k):
    return np.cumsum(delta * forward_rates * sigma_j * sigma_k/(1 + delta * forward_rates), axis=1)

def getInitialNextForwardEstimate(forward_rates, mu, sigma, delta, Z):
    return forward_rates * np.exp((mu - (sigma**2)/2) * delta + sigma * np.sqrt(delta) * Z)


However, to improve the accuracy, we will use the Predictor-Corrector method which involves estimating the next period's drift and then taking the average. Hence, the final estimate of the forward rate is given by:

$F_j(t_i) = F_j(t_{i-1})exp[\frac{1}{2}(\mu_j(t_{i-1})+\mu_j(t_i)-\sigma^2)\delta_{i-1} + \sigma_j\sqrt{\delta_{i-1}}Z_i]$

This final estimate is written as:

In [6]:
def getFinalNextForward(forward_rates, mu1, mu2, sigma, delta, Z):
    return forward_rates * np.exp(0.5 * (mu1 + mu2 - sigma**2) * delta + sigma * np.sqrt(delta) * Z)

Furthermore, the continuously compounded interest rate can be derived from the LIBOR forward rate as follows:

$e^{r_{ti}(t_{i+1}-t_i)} = 1 + L(t_i, t_{i+1})(t_{i+1} - t_i)$

In [7]:
def liborToContinuousRate(libor, dt):
    return 1 + (libor * dt)

## Volatility - CEV Model

We will be using the CEV model to represent volatility, which is given by

$\sigma(t_i, t_{i+1}) = \sigma(S_{it})^{\gamma-1}$

In [8]:
def getVolatility(S_t, gamma, sigma):
    return sigma * S_t**(gamma - 1)

## Stock Price Model

We assume the following lognormal evolution of stock price/firm value:

$S_{t_{i+1}} = S_{t_i}exp[(r - \frac{\sigma^2(t_i, t_{i+1})}{2})(t_{i+1}-t_i)+\sigma(t_i, t_{i+1})\sqrt{t_{i+1}-t_i}Z]$

which can be re-written as

$S_{t_{i+1}} = S_{t_i}exp[r(t_{i+1}-t_i)]exp[(- \frac{\sigma^2(t_i, t_{i+1})}{2})(t_{i+1}-t_i)+\sigma(t_i, t_{i+1})\sqrt{t_{i+1}-t_i}Z]$

In [9]:
def getSharePrice(S_t, continuous_rate, sigma, dt, Z):
    return S_t * continuous_rate * np.exp(-sigma**2/2 * dt + sigma * np.sqrt(dt) * Z)

## Simulation

First, we initialise the stock matrix and delta matrix.

In [10]:
delta = np.ones([n_simulations, n_steps - 1]) * dt
stock = np.ones([n_simulations, n_steps]) * S_0
firm = np.ones([n_simulations, n_steps]) * V_0

continuesRates = []
volatilities = []

We also need to define a method to reshape our data structure

In [11]:
def reshape(input):
    return np.array([input, ]).transpose()

Then, we go to the actual simulation.

In [12]:
print("Values for 1st Simulation only\n")

print("Values for 1st Simulation only\n")

for i in range(0, 12):
    #each iteration is a point in time, estimating for 12 forward rates
    forward_Z = norm.rvs(size = [n_simulations, 1])
    stock_Z = norm.rvs(size = [n_simulations, 1])
    firm_Z = norm.rvs(size = [n_simulations, 1])
    
    S_t_minus_one = stock[:, i]
    V_t_minus_one = firm[:, i]
    forward_rates_t = forward_rates[:, i:]
    delta_t = delta[:, i:]
    
    sigma_new = getVolatility(S_t_minus_one, gamma, sigma_s)
    volatilities.append(sigma_new)
    #duplicate the sigma across more columns as the same sigma will be used to calculate the other forward rates 
    sigma_new_transposed = np.array([sigma_new,] * (n_steps - (i + 1))).transpose() 
    
    mu_initial = getMu(forward_rates_t, delta_t, sigma_new_transposed, sigma_new_transposed)
    forward_temp = getInitialNextForwardEstimate(forward_rates_t, mu_initial, sigma_new_transposed, delta_t, forward_Z)
    mu_next = getMu(forward_temp, delta_t, sigma_new_transposed, sigma_new_transposed)
    final_forwards = getFinalNextForward(forward_rates_t, mu_initial, mu_next, sigma_new_transposed, delta_t, forward_Z)
    forward_rates[:,i:] = final_forwards
    
    first_libor_rate = forward_rates[:,i]
    continuous_rate = liborToContinuousRate(first_libor_rate, dt)
    continuesRates.append(continuous_rate)
    
    continuous_rate = reshape(continuous_rate)
    sigma_new = reshape(sigma_new)
    S_t_minus_one = reshape(S_t_minus_one)
    V_t_minus_one = reshape(V_t_minus_one)
    S_t = getSharePrice(S_t_minus_one, continuous_rate, sigma_new, dt, stock_Z)
    V_t = getSharePrice(V_t_minus_one, continuous_rate, sigma_new, dt, firm_Z)
    stock[:, i + 1] = S_t[:,0]
    firm[:, i + 1] = V_t[:,0]
    
    print("Volatility at time " + str(i+1) + ": " + str(sigma_new[0]))
    print("Forward Rates at time " + str(i+1) + ": " + str(final_forwards[0]))
    print("First LIBOR rate at time " + str(i+1) + ": " + str(first_libor_rate[0]))
    print("Continuos rate at time " + str(i+1) + ": " + str(continuous_rate[0]))
    print("Stock Price at time " + str(i+1) + ": " + str(S_t[0]))
    print("Firm Value at time " + str(i+1) + ": " + str(V_t[0]))
    print("\n")

Values for 1st Simulation only

Values for 1st Simulation only

Volatility at time 1: [0.09486833]
Forward Rates at time 1: [0.0773802  0.07786635 0.07708694 0.07756939 0.07677036 0.07724885
 0.07773333 0.07560056 0.07738417 0.07654237 0.07568199 0.07614695]
First LIBOR rate at time 1: 0.07738019776669464
Continuos rate at time 1: [1.00644835]
Stock Price at time 1: [104.21512167]
Firm Value at time 1: [204.6763797]


Volatility at time 2: [0.09389416]
Forward Rates at time 2: [0.07992317 0.07912355 0.07961913 0.07879936 0.07929087 0.07978854
 0.07759975 0.0794309  0.0785672  0.07768443 0.07816206]
First LIBOR rate at time 2: 0.07992317308265562
Continuos rate at time 2: [1.00666026]
Stock Price at time 2: [108.09401156]
Firm Value at time 2: [207.31570857]


Volatility at time 3: [0.09304025]
Forward Rates at time 3: [0.07617041 0.07664785 0.07585902 0.07633255 0.07681201 0.07470521
 0.07646842 0.07563728 0.07478777 0.07524793]
First LIBOR rate at time 3: 0.07617040930531864
Continuos

# Question 2

In [13]:
def simulateLIBOR(n):

  continuous_rates = np.zeros(n)

  for i in range(0, n):
    forward_Z = norm.rvs(size = [n_simulations, 1])
    stock_Z = norm.rvs(size = [n_simulations, 1])
    
    S_t_minus_one = stock[:, i]
    forward_rates_t = forward_rates[:, i:]
    delta_t = delta[:, i:]
    
    sigma_new = getVolatility(S_t_minus_one, gamma, sigma_s)
    #duplicate the sigma across more columns as the same sigma will be used to calculate the other forward rates 
    sigma_new_transposed = np.array([sigma_new,] * (n_steps - (i + 1))).transpose() 
    
    mu_initial = getMu(forward_rates_t, delta_t, sigma_new_transposed, sigma_new_transposed)
    forward_temp = getInitialNextForwardEstimate(forward_rates_t, mu_initial, sigma_new_transposed, delta_t, forward_Z)
    mu_next = getMu(forward_temp, delta_t, sigma_new_transposed, sigma_new_transposed)
    final_forwards = getFinalNextForward(forward_rates_t, mu_initial, mu_next, sigma_new_transposed, delta_t, forward_Z)
    forward_rates[:,i:] = final_forwards
    
    first_libor_rate = forward_rates[:,i]
    continuous_rate = liborToContinuousRate(first_libor_rate, dt)
    
    continuous_rate = reshape(continuous_rate)
    continuous_rates[i] = continuous_rate[0]
    sigma_new = reshape(sigma_new)
    S_t_minus_one = reshape(S_t_minus_one)
    S_t = getSharePrice(S_t_minus_one, continuous_rate, sigma_new, dt, stock_Z)
    stock[:, i + 1] = S_t[:,0]

  return np.prod(continuous_rates)

In [14]:
def share_path(S_0, r, sigma, Z, dT):
    return S_0 * np.exp(np.cumsum((r - sigma**2/2) * dT + sigma * np.sqrt(dT) * Z))

def european_call_payoff(S_T, K, r, T):
    """
    This function takes in the following parameters to determine the price of the European call option:
      S_T is the price of the underlying at expiry.
      K is the strike price
      r is the risk free rate
      T is the time period till maturity. This can be in months or years depending on the user's preference. In our case, it is in year.

    The value of the European call option is determined by taking the discounted value of the final call option payoff
    using the formula: np.exp(-T*r) * np.maximum(0, S_T - K)
    """
    return np.exp(-T*r) * np.maximum(0, S_T - K) #payoff for call option

def euro_uao_call(barrier, paths, K, r, T):
    """
    This functions takes in the following parameters to calculate the value of a European Up-And-Out call option:
      barrier is the barrier level,
      paths is a list of share price paths,
      K is the strike price
      r is the risk free rate
      T is the time period of each share price path.
    """
    prices = []
    for path in paths:
            if np.max(path) > barrier: # knocked out
                    prices.append(0)
            else:
                    prices.append(european_call_payoff(path[-1], K, r, T)) 
    return np.mean(prices)

In [ ]:
call_val_list = []
cva_list = []
adjusted_call_list = []
corr_matrix = np.array([[1, correlation], [correlation, 1]])

for sampleSize in tqdm(range(1000, n_simulations + 1000, 1000)):
    share_path_list = []
    firm_value_list = []
    r_list = []

    r = simulateLIBOR(12) - 1
    r_list.append(r)
    
    #for each sample size, sum up all price path for each simulation so that the mean can be calculated later
    for i in range(0, sampleSize):
        norm_matrix = norm.rvs(size=np.array([2, frequency]))
        corr_norm_matrix = np.matmul(np.linalg.cholesky(corr_matrix), norm_matrix)

        share_price_path = share_path(S_0, r, sigma_s, corr_norm_matrix[0,], T/frequency)
        firm_value_path = share_path(V_0, r, sigma_v, corr_norm_matrix[1,], T/frequency)
        share_path_list.append(share_price_path)
        firm_value_list.append(firm_value_path)

    #get the mean path for the sum of all the simulations
    share_path_mean = list(map(lambda summed: summed/sampleSize,  functools.reduce(lambda a,b: [x + y for x, y in zip(a, b)], share_path_list)))
    firm_value_mean = list(map(lambda summed: summed/sampleSize,  functools.reduce(lambda a,b: [x + y for x, y in zip(a, b)], firm_value_list)))

    ################################################
    #######     terminal value of option   #########
    ################################################
    #print(share_path_list)
    call_val = euro_uao_call(L, share_path_list, K, r, T) # obtain the Up-and-out European call option value using the defined function euro_uao_call
    # Extract the terminal firm values
    term_firm_values = list(map(lambda x: x[-1], firm_value_list))
    
    # Note: the explanation of the functions are defined above under the "Define functions" section

    ################################################
    #######               CVA              #########
    ################################################

    # To calculate the amount lost, we apply the formula [np.exp(-T/frequency*r) * (1-recovery_rate)*(term_firm_val < debt) * call_val]
    # for each simulated terminal firm value in the monte carlo simulation. 
    amount_lost = [np.exp(-T/frequency*r) * (1-recovery_rate)*(term_firm_val < debt) * call_val for term_firm_val in term_firm_values] # calculate the amount lost
    cva = np.mean(amount_lost)
    adjusted_opt_val = call_val - cva
    
    call_val_list.append(call_val)
    cva_list.append(cva)
    adjusted_call_list.append(adjusted_opt_val)

  6%|████▉                                                                             | 6/100 [00:14<03:49,  2.44s/it]

In [ ]:
print("The mean value of the call option with no default risk based on simulations is {}".format(np.mean(call_val_list)))
print("The mean value of the CVA based on simulations is {}".format( np.mean(cva_list)))
print("The mean adjusted value of the  call option based on simulations is {}".format(np.mean(adjusted_call_list)))

In [ ]:
# Reprint values for 1st simulation only

print("Number of simulations: {}".format(1000))
print("One-year discount factor based on LFMM: {}".format(math.pow(r_list[0] + 1, -1)))

print('\n')

print("The mean value of the call option with no default risk based on {} simulations is {}".format(1000, np.mean(call_val_list[0])))
print("The mean value of the CVA based on {} simulations is {}".format(1000, np.mean(cva_list[0])))
print("The mean adjusted value of the  call option based on {} simulations is {}".format(1000, np.mean(adjusted_call_list[0])))